In [1]:
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
# from torch.distributions.categorical import Categorical
from EPMS.serialization import SETTINGS
import os

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
class EPMSDataset(Dataset):
    def __init__(self, table_dir, instrument):
        self.instrument = instrument
        self.dataframes = []
        for filepath in os.listdir(table_dir):
            if not filepath.endswith('.pkl'):
                continue
            self.dataframes.append(pd.read_pickle(f'{table_dir}/{filepath}'))

    def get_dataframe_notes(self, dataframe):
        dataframe_instrument = dataframe[dataframe.INSTRUMENT == self.instrument]
        initial_note = len(dataframe.columns) - SETTINGS["KEYBOARD_SIZE"]
        dataframe_np = dataframe_instrument.iloc[:, initial_note:].astype("float").to_numpy()
        return dataframe_np

    def __len__(self):
        return len(self.dataframes)

    def __getitem__(self, idx):
        notes = np.ceil(self.get_dataframe_notes(self.dataframes[idx]))
        return torch.tensor(notes[:-1]), torch.tensor(notes[1:]).float()


In [4]:
batch_size = 1
dataset_name = 'C_major_scale'
dataset = EPMSDataset(dataset_name + "/", "Piano")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [5]:
class PapagaioLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, n_layers):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim*vocab_size, hidden_size, n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, h0=None, c0=None):
        x = self.embedding_layer(x.long())
        x = x.flatten(2, 3)
        if h0 is None or c0 is None:
            out, (hf, cf) = self.lstm(x)
        else:
            out, (hf, cf) = self.lstm(x, (h0, c0))
        out = self.linear(out)
        out = self.sigmoid(out)
        return out, (hf, cf)

In [6]:
vocab_size = SETTINGS["KEYBOARD_SIZE"]
embedding_dim = 64
hidden_size = 1024
n_layers = 1
epochs = 100
lr = 0.001

In [7]:
papagaio_lstm = PapagaioLSTM(vocab_size, embedding_dim, hidden_size, n_layers).to(device)
optimizer = optim.Adam(papagaio_lstm.parameters(), lr)
loss_fn = nn.BCELoss()

for epoch in tqdm(range(epochs)):
    total_loss = 0
    for batch in dataloader:
        x = batch[0].to(device)
        y = batch[1].to(device)
        optimizer.zero_grad()
        output, _ = papagaio_lstm(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()/batch_size

    print(f'Epoch: {epoch+1}')
    print(f'Loss: {total_loss}')

  1%|          | 1/100 [00:01<02:11,  1.33s/it]

Epoch: 1
Loss: 0.7222133278846741


  2%|▏         | 2/100 [00:01<01:08,  1.43it/s]

Epoch: 2
Loss: 0.30464601516723633


  3%|▎         | 3/100 [00:01<00:47,  2.05it/s]

...

Epoch: 98
Loss: 0.03314445912837982
Epoch: 99

 99%|█████████▉| 99/100 [00:24<00:00,  4.39it/s]


Loss: 0.03314301371574402


100%|██████████| 100/100 [00:24<00:00,  4.08it/s]

Epoch: 100
Loss: 0.03314156457781792


In [8]:
# torch.save(papagaio_lstm.state_dict(), f'models/papagaio_model_{dataset_name}_{epochs}epochs_{lr}lr.pt')

In [9]:
# papagaio_lstm.load_state_dict(torch.load('models/papagaio_model.pt', map_location=device))